In [2]:
#read excell file
import pandas as pd
import fitter
import matplotlib.pyplot as plt
import scipy.stats as stats
import dill


In [12]:
#read excell file
#header is on row 3
df = pd.read_excel('./je-d-01.05.04.01.01.xlsx', header=2,sheet_name="Einwanderungen")
#cut off after row 49
df = df[:49]
#cut off row 2 to 24
df = df.drop(df.index[2:25])
#set first column as index
df = df.set_index(df.columns[0])

#replace * with nan
df = df.apply(pd.to_numeric, errors='coerce')
#keep only the following indexes
df = df.loc[['Total','Deutschland', 'Österreich', 'Schweizer', 'Frankreich', 'Italien']]
#Transpose the dataframe
df = df.T

df["Deutsch"] = df["Deutschland"] + df["Österreich"] + df["Schweizer"]
df["Andere"] = df["Total"] - df["Deutsch"] - df["Frankreich"] - df["Italien"]
#drop Columns Total Deutschland Österreich Schweizer 
df = df.drop(df.columns[0:4], axis=1)
#rename Frankreich to Französisch
df = df.rename(index=str, columns={"Frankreich": "Französisch"})
#rename Italien to Italienisch
df = df.rename(index=str, columns={"Italien": "Italienisch"})

In [13]:
#for each column find the best fit distribution
dict_distributions = {}
for col in df.columns:
    #fit the data and drop na
    f = fitter.Fitter(df[col].dropna(), timeout=60)
    f.fit()
    dict_distributions[col] = f.get_best()

print(dict_distributions)
    

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/integrate/_quadpack_py.py:879: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/integrate/_quadpack_py.py:879: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/integrate

{'Französisch': {'powerlaw': {'a': 0.5972845912678386, 'loc': 4879.999999999999, 'scale': 11018.251678052946}}, 'Italienisch': {'arcsine': {'loc': 4313.999999999999, 'scale': 14693.255218905497}}, 'Deutsch': {'nakagami': {'nu': 0.43359957710345587, 'loc': 31488.999999999996, 'scale': 20261.703914740316}}, 'Andere': {'kappa4': {'h': 1.0673671474698967, 'k': 1.0628748448970646, 'loc': 46775.03913456213, 'scale': 66729.36691718586}}}


In [14]:
dill.dump(dict_distributions, open("distributions_nach_sprache.pick", mode='wb'))